In [1]:
# If we are on colab: this clones the repo and installs the dependencies
from pathlib import Path
# if Path.cwd().name != 'LightGlue':
#   !git clone --quiet https://github.com/cvg/LightGlue/
#   %cd LightGlue
#   !pip install --progress-bar off --quiet -e .
import importlib
from lightglue import LightGlue, SuperPoint, DISK
from lightglue.utils import load_image, rbd
from lightglue import viz2d
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import torch
import torch.nn.functional as F

torch.set_grad_enabled(False)
images = Path('assets')
%load_ext autoreload
%autoreload 2

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # 'mps', 'cpu'
# importlib.reload(LightGlue)

extractor = SuperPoint(max_num_keypoints=2048).eval().to(device)  # load the extractor
matcher = LightGlue(features='superpoint').eval().to(device)

In [3]:
image0 = load_image(images / 'DSC_0411.JPG')
image1 = load_image(images / 'DSC_0410.JPG')
print(image0.shape)

feats0 = extractor.extract(image0.to(device))
feats1 = extractor.extract(image1.to(device))

torch.Size([3, 682, 1024])


In [4]:
print(image0.shape)

torch.Size([3, 682, 1024])


In [5]:
from torchrl.modules import MLP
from torch import nn

class MLP_module(nn.Module):
    def __init__(self):
        super().__init__()
        self.MLP = MLP(in_features=256, out_features=3, num_cells=[128, 64, 32, 16])
        self.MLP_de = MLP(in_features=3, out_features=256, num_cells=[16, 32, 64, 128])
    def forward(self, desc0: torch.Tensor, desc1: torch.Tensor):
        desc0_mlp = self.MLP(desc0)
        desc1_mlp = self.MLP(desc1)
        desc0_back = self.MLP_de(desc0_mlp)
        desc1_back = self.MLP_de(desc1_mlp)
        return desc0_mlp, desc1_mlp

model = MLP_module().to(device)
checkpoint = torch.load("/mnt/home_6T/public/koki/MLP_data/MLP_checkpoint/model_20231212_171923_7")

model.load_state_dict(checkpoint)

feats0['descriptors'] , feats1['descriptors'] = model(feats0['descriptors'],feats1['descriptors'])

In [6]:
print(feats0['descriptors'])

tensor([[[ 0.0797,  0.6430,  0.3348],
         [ 0.2118, -1.3800, -2.4669],
         [-0.0219, -0.5547, -0.0225],
         ...,
         [-0.5071, -1.5036, -0.8785],
         [ 0.3146,  1.2901, -1.3160],
         [-0.8707, -0.4054, -2.6474]]], device='cuda:0')


In [15]:
from Transformer_model import myGlue
matcher = myGlue().to(device)
matches01 = matcher({"image0": feats0, "image1": feats1})

In [16]:
matches01['stop']

6